In [66]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
import re

import pandas as pd
from bs4 import BeautifulSoup
import re

import time

In [22]:
url = 'https://www.lme.com/Metals/Non-ferrous/LME-Aluminium#Trading+day+summary'

service = Service()
driver = webdriver.Chrome(service=service)

try:
    driver.maximize_window()
    driver.get(url=url)
    
    time.sleep(2)
    
    # Define the JavaScript code to scroll to the bottom of the page
    scroll_to_bottom = "window.scrollTo(0, document.body.scrollHeight);"

    
    driver.execute_script("window.scrollTo(0, window.scrollY + window.innerHeight);")
    driver.execute_script("window.scrollTo(0, window.scrollY + window.innerHeight);")
    
    time.sleep(2)
    
    html_code = driver.page_source
    
    with open("page.html", "w", encoding="utf-8") as file:
        file.write(html_code)


except Exception as error:
    print(error)


finally:
    driver.close()
    driver.quit()

In [48]:
soup = BeautifulSoup(html_code, 'html.parser')
soup

<html lang="en-GB" prefix="og: http://ogp.me/ns#"><head>
<meta charset="utf-8"/>
<meta content="width=device-width, height=device-height, initial-scale=1.0, shrink-to-fit=no" name="viewport"/>
<meta content="IE=edge, chrome=1" http-equiv="X-UA-Compatible"/>
<meta content="telephone=no" name="format-detection"/>
<meta content="2023 The London Metal Exchange" name="copyright"/>
<meta content="The London Metal Exchange" name="author"/>
<title>LME Aluminium | London Metal Exchange </title>
<link href="https://www.lme.com/Metals/Non-ferrous/LME-Aluminium" rel="canonical"/>
<meta content="Current and historical Aluminium prices, stocks and monthly averages." name="description"/>
<meta content="index,follow" name="robots"/>
<meta content="https://www.lme.com/Metals/Non-ferrous/LME-Aluminium" property="og:url"/>
<link href="https://www.lme.com/en/metals/non-ferrous/lme-aluminium" hreflang="en" rel="alternate"/>
<meta content="LME Aluminium | London Metal Exchange " property="og:title"/>
<meta 

In [89]:
date_raw = (soup.find_all('span', class_='data-set-tabs__content-detail')[2].text)
date_raw = date_raw.strip()
date_list = re.sub(r'\W+', ' ', date_raw)
date_list = date_list.split(' ')
date = date_list[3:]
date = ' '.join(date)
date = pd.to_datetime(date)
date


Timestamp('2023-11-07 00:00:00')

In [24]:
data_raw = soup.find_all('td', class_='data-set-table__cell')


In [25]:
price = data_raw[1].text
pd.to_numeric(price)

2241.0

In [127]:
def lme_selenium():
    def get_data(metall=''):
        url = f'https://www.lme.com/Metals/Non-ferrous/LME-{metall}#Trading+day+summary'

        service = Service()
        driver = webdriver.Chrome(service=service)

        try:
            driver.maximize_window()
            driver.get(url=url)

            time.sleep(2)

            driver.execute_script(
                "window.scrollTo(0, window.scrollY + window.innerHeight);")
            driver.execute_script(
                "window.scrollTo(0, window.scrollY + window.innerHeight);")

            time.sleep(2)

            html_code = driver.page_source

            soup = BeautifulSoup(html_code, 'html.parser')
            data_raw = soup.find_all('td', class_='data-set-table__cell')
            price = pd.to_numeric(data_raw[1].text)

            # Date extraction
            date_raw = (soup.find_all(
                'span', class_='data-set-tabs__content-detail')[2].text)
            date_raw = date_raw.strip()
            date_list = re.sub(r'\W+', ' ', date_raw)
            date_list = date_list.split(' ')
            date = date_list[3:]
            date = ' '.join(date)
            date = pd.to_datetime(date)

            return (date, price)

        except Exception as error:
            print(error)

        finally:
            driver.close()
            driver.quit()

    date, _ = get_data('Aluminium')
    _, al_price = get_data('Aluminium')
    _, cu_price = get_data('Copper')
    _, zn_price = get_data('Zinc')
    _, nk_price = get_data('Nickel')
    _, ld_pric = get_data('Lead')

    data_row = pd.DataFrame({
        'date': date,
        'aluminium': al_price,
        'copper': cu_price,
        'lead': ld_pric,
        'nickel': nk_price,
        'zink': zn_price
    }, index=[0])

    historical_data = pd.read_excel(
        '../parser_beta/data/LME_db.xlsx', index_col=0)

    historical_data = pd.concat(
        [historical_data, data_row]).reset_index(drop=True)

    historical_data.drop_duplicates(inplace=True)

    with pd.ExcelWriter('/test.xlsx') as writer:
        historical_data.to_excel(writer, sheet_name='LME_non_ferrous')

    return historical_data

In [128]:
lme_selenium()

OSError: [Errno 30] Read-only file system: '/test.xlsx'

In [41]:
al_price = 2089.0
cu_price = 8132.0
zn_price = 2282.0
nk_price = 19850
ld_pric = 2123.0

In [ ]:
date_raw = (soup.find_all('span', class_='data-set-tabs__content-detail')[2].text)
date_raw = date_raw.strip()
date_list = re.sub(r'\W+', ' ', date_raw)
date_list = date_list.split(' ')
date = date_list[3:]
date = ' '.join(date)
date = pd.to_datetime(date)
date


In [91]:
new_data = pd.DataFrame({
    'date':date,
    'aluminium':al_price,
    'copper':cu_price,
    'lead':ld_pric,
    'nickel':nk_price,
    'zink':zn_price
}, index=[0])

new_data

,date,aluminium,copper,lead,nickel,zink
0,2023-11-07,2089.0,8132.0,2123.0,19850,2282.0


In [99]:
new_hist = pd.concat([historical_data, new_data]).reset_index(drop=True)
new_hist

,date,aluminium,copper,lead,nickel,zink
0,2022-01-04,2815.5,9660.0,2327.0,20730,3602.0
1,2022-01-05,2866.0,9778.0,2343.0,20900,3660.0
2,2022-01-06,2912.5,9565.0,2291.0,20480,3590.0
3,2022-01-07,2919.5,9615.0,2342.0,20725,3602.5
4,2022-01-10,2923.0,9665.0,2303.0,21045,3576.5
...,...,...,...,...,...,...
404,2023-08-11,2127.0,8242.0,2118.0,20000,2415.5
405,2023-08-14,2096.0,8242.0,2088.0,19825,2357.0
406,2023-08-15,2085.5,8126.5,2099.0,19685,2313.0
407,2023-08-16,2089.0,8132.0,2123.0,19850,2282.0


In [102]:
new_hist = pd.concat([new_hist, new_data]).reset_index(drop=True)
new_hist

,date,aluminium,copper,lead,nickel,zink
0,2022-01-04,2815.5,9660.0,2327.0,20730,3602.0
1,2022-01-05,2866.0,9778.0,2343.0,20900,3660.0
2,2022-01-06,2912.5,9565.0,2291.0,20480,3590.0
3,2022-01-07,2919.5,9615.0,2342.0,20725,3602.5
4,2022-01-10,2923.0,9665.0,2303.0,21045,3576.5
...,...,...,...,...,...,...
406,2023-08-15,2085.5,8126.5,2099.0,19685,2313.0
407,2023-08-16,2089.0,8132.0,2123.0,19850,2282.0
408,2023-11-07,2089.0,8132.0,2123.0,19850,2282.0
409,2023-11-07,2089.0,8132.0,2123.0,19850,2282.0


In [104]:
new_hist.drop_duplicates()

,date,aluminium,copper,lead,nickel,zink
0,2022-01-04,2815.5,9660.0,2327.0,20730,3602.0
1,2022-01-05,2866.0,9778.0,2343.0,20900,3660.0
2,2022-01-06,2912.5,9565.0,2291.0,20480,3590.0
3,2022-01-07,2919.5,9615.0,2342.0,20725,3602.5
4,2022-01-10,2923.0,9665.0,2303.0,21045,3576.5
...,...,...,...,...,...,...
404,2023-08-11,2127.0,8242.0,2118.0,20000,2415.5
405,2023-08-14,2096.0,8242.0,2088.0,19825,2357.0
406,2023-08-15,2085.5,8126.5,2099.0,19685,2313.0
407,2023-08-16,2089.0,8132.0,2123.0,19850,2282.0
